In [ ]:
# Clean and install correct versions
%pip uninstall -y numpy transformers datasets tokenizers huggingface-hub
%pip install numpy==1.26.4 transformers==4.37.2 datasets==2.19.1 --no-cache-dir --force-reinstall
%pip install peft==0.10.0
%pip install accelerate==0.27.2
%pip install kagglehub[pandas-datasets]
%pip install pandas, numpy, scikit-learn, kagglehub, nltk, torch, datasets

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter
# # data  used https://www.kaggle.com/datasets/andrewmvd/cyberbullying-classification
# # https://www.kaggle.com/datasets/saniyatlamim007/cyber-bulling-large-dataset?resource=download
# # https://www.kaggle.com/datasets/saurabhshahane/cyberbullying-dataset

# Load the latest version
df1 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "saurabhshahane/cyberbullying-dataset",
  "aggression_parsed_dataset.csv",
  pandas_kwargs={"encoding": "latin-1"},
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)
df2 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "saurabhshahane/cyberbullying-dataset",
  "attack_parsed_dataset.csv",
  pandas_kwargs={"encoding": "latin-1"},)

df3 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "saurabhshahane/cyberbullying-dataset",
  "kaggle_parsed_dataset.csv",
  pandas_kwargs={"encoding": "latin-1"},)

df4 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "saurabhshahane/cyberbullying-dataset",
  "toxicity_parsed_dataset.csv",
  pandas_kwargs={"encoding": "latin-1"},)

df5 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "saurabhshahane/cyberbullying-dataset",
  "twitter_parsed_dataset.csv",
  pandas_kwargs={"encoding": "latin-1"},)

df6 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "saurabhshahane/cyberbullying-dataset",
  "twitter_parsed_dataset.csv",
  pandas_kwargs={"encoding": "latin-1"},)

df7 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "saurabhshahane/cyberbullying-dataset",
  "twitter_parsed_dataset.csv",
  pandas_kwargs={"encoding": "latin-1"},)

df8 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "saurabhshahane/cyberbullying-dataset",
  "twitter_parsed_dataset.csv",
  pandas_kwargs={"encoding": "latin-1"},)

df9 = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "andrewmvd/cyberbullying-classification",
    "cyberbullying_tweets.csv",
    pandas_kwargs={"encoding": "latin-1"},
)

df9["label"] = df9["cyberbullying_type"].apply(lambda x: 0 if x == "not_cyberbullying" else 1)
df9 = df9[["tweet_text", "label"]]

df10 = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "saifulislam7/cyberbullying-and-harassment-detection-using-ml",
    "Approach to Social Media Cyberbullying and Harassment Detection Using Advanced Machine Learning.csv",
    pandas_kwargs={"encoding": "latin-1"},
)

df10 = df10[['Text', 'Label']]

# print("First 5 records df1:", df1.head())
# print("First 5 records: df2", df2.head())
# print("First 5 records: df3", df3.head())
# print("First 5 records: df4", df4.head())
# print("First 5 records: df5", df5.head())
# print("First 5 records: df6", df6.head())
# print("First 5 records: df7", df7.head())
# print("First 5 records: df8", df8.head())
# print("First 5 records: df9", df9.head())
# print("First 5 records: df10", df10.head())

In [3]:
df10 = df10.dropna(subset=["Label"])
df10["label"] = df10["Label"].apply(lambda x: 0 if 'not' in x.lower() else 1)
df10 = df10[["Text", "label"]]
df10.head()

,Text,label
0,Ten outside soon doctor shake everyone treatme...,0
1,my life has come to a standstill and at this p...,0
2,girl this nigga make me sick to my stomach,1
3,I wanna fuck you,1
4,"Oh hey, you should be ashamed of your disgusti...",0


In [4]:
dfs = [df1, df2, df3, df4, df5, df6, df7, df8]
df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all.rename(columns={"Text": "text", "tweet_text": "text", "oh_label": "label"})
df_all = df_all[["text", "label"]]
df9 = df9.rename(columns={"tweet_text": "text"})
df9 = df9[["text", "label"]]
df10 = df10.rename(columns={"Text": "text"})
df10 = df10[["text", "label"]]
df_all = pd.concat([df_all, df9, df10], ignore_index=True)
print(df_all.shape)            # total rows, should be sum of all dfs
print(df_all["label"].value_counts(normalize=True))  # class balance

(523759, 2)
label
0.0    0.785184
1.0    0.214816
Name: proportion, dtype: float64


In [6]:
import re
import unicodedata
import html
import string

def preprocess_text(text):
    # Check if input is valid
    if not isinstance(text, str):
        if text is None:
            return ""
        try:
            text = str(text)
        except:
            return ""

    # Handle empty strings
    if not text or text.isspace():
        return ""

    # Decode HTML entities
    text = html.unescape(text)

    # Normalize unicode characters
    text = unicodedata.normalize('NFKC', text)

    # Replace URLs with [URL] token
    text = re.sub(r'https?://\S+|www\.\S+', '[URL]', text)

    # Replace emails with [EMAIL] token
    text = re.sub(r'\S+@\S+', '[EMAIL]', text)

    # Replace phone numbers with [PHONE] token
    text = re.sub(r'\(?[\d]{3}\)?[-.\s]?[\d]{3}[-.\s]?[\d]{4}', '[PHONE]', text)

    # Replace numbers with [NUMBER] token (optional, depending on your use case)
    # text = re.sub(r'\b\d+\b', '[NUMBER]', text)

    # Remove excessive whitespace (including newlines and tabs)
    text = re.sub(r'\s+', ' ', text)

    # Remove excessive punctuation (e.g., "!!!" becomes "!")
    # Escape punctuation characters before using them in re.sub
    for punct in string.punctuation:
        # Escape the punctuation character to use it literally in the regex
        escaped_punct = re.escape(punct)
        # Create a pattern to match one or more consecutive occurrences of the escaped punctuation
        pattern = escaped_punct + r'+'  # Use r'+' for clarity and safety
        # Replace consecutive occurrences with a single instance of the punctuation
        text = re.sub(pattern, lambda m: m.group(0)[0], text)

    # Strip leading and trailing whitespace
    text = text.strip()

    return text

In [7]:
df_all["text"] = df_all["text"].apply(preprocess_text)
df_all.head()

,text,label
0,`- This is not `creative`. Those are the dicti...,0.0
1,` : the term `standard model` is itself less N...,0.0
2,"True or false, the situation as of March 2002 ...",0.0
3,"Next, maybe you could work on being less conde...",0.0
4,This page will need disambiguation.,0.0


In [8]:
from sklearn.model_selection import train_test_split
df_all = df_all.dropna(subset=["label"])


df_trainval, df_test = train_test_split(
    df_all,
    test_size=0.15,
    random_state=42,
    stratify=df_all["label"]
)

df_train, df_val = train_test_split(
    df_trainval,
    test_size=0.1765,
    random_state=42,
    stratify=df_trainval["label"]
)

print("Train:", df_train.shape)
print("Validation:", df_val.shape)
print("Test:", df_test.shape)


Train: (366609, 2)
Validation: (78575, 2)
Test: (78563, 2)


In [9]:
# How many NaNs are in your label column?
print("Missing labels:", df_all["label"].isnull().sum())

# Peek at the examples with missing labels
df_all[df_all["label"].isnull()].head()



Missing labels: 0


,text,label


In [37]:
from datasets import Dataset, DatasetDict
ds_train = Dataset.from_pandas(df_train)
ds_val   = Dataset.from_pandas(df_val)
ds_test  = Dataset.from_pandas(df_test)

datasets = DatasetDict({
    "train": ds_train,
    "validation": ds_val,
    "test": ds_test
})


In [38]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_datasets = datasets.map(
    tokenize_fn,
    batched=True,
    remove_columns=["text"]   # we no longer need raw text
)

Map:   0%|          | 0/366609 [00:00<?, ? examples/s]

Map:   0%|          | 0/78575 [00:00<?, ? examples/s]

Map:   0%|          | 0/78563 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
import torch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets.set_format(type="numpy")

tokenized_datasets.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"],
    output_all_columns=True # Keep other columns if necessary
)

from datasets import ClassLabel, Value

# Cast the label column to int64
tokenized_datasets = tokenized_datasets.cast_column("label", Value("int64"))
print(tokenized_datasets["train"][0]["label"].dtype)  # should say torch.int64

In [40]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2  # change this if you have more than two classes
)

# Force the model to treat this as a classification task, not multi-label
model.config.problem_type = "single_label_classification"

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

In [42]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "f1": f1_score(p.label_ids, preds, average="weighted")
    }

In [43]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [44]:
print(tokenized_datasets["train"][0])
print(model.config.problem_type)

{'label': tensor(1), 'input_ids': tensor([ 101, 6616, 2125, 2017, 5390, 3676, 3762, 4485, 1997, 1037, 2158,  102,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0,

In [45]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 